In [1]:
import numpy as np
import random

def do_exp(theta, plan):
    rss_avg, norm_avg_hat = 0, 0
    for i in range(100):
        X, Y = create_X(plan), np.log(create_Y(plan, theta))
        theta_hat = calculate_least_squares(X, Y)
        Y_hat = np.dot(X, theta_hat)
        rss_avg += np.dot(Y - Y_hat, Y - Y_hat)
        norm_avg_hat += np.dot(theta_hat - theta, theta_hat - theta)
    return rss_avg / 100, norm_avg_hat / 100

def make_opt(theta_hat, plan, grid, N):
    eps = 0.0001
    iteration = 0
    while True:
        M = create_M(plan, N, theta_hat)
        print("det", np.linalg.det(M))
        D = create_D(M)
        print(iteration)
        delta = find_max(plan, D, N, grid, theta_hat)
        if delta[0] > eps:
            plan[delta[2]] = delta[1]
        else:
            break
        iteration += 1
    return plan

# d nu(x, theta) / d theta
def func(x, theta):
  return np.array([x[0]**theta[1] * x[1]**theta[2] * x[2]**theta[3],
                     theta[0] * x[1]**theta[2] * x[2]**theta[3] * theta[1] * x[0]**(theta[1]-1), 
                     theta[3] * theta[0] * x[0]**theta[1] * x[1]**theta[2] * x[2]**(theta[3]-1)])

def make_plan(grid = np.linspace(0, 20, 1001), m = 10):
    return list(map(lambda x: [random.choice(grid), random.choice(grid),  random.choice(grid)], range(m)))

def calculate_least_squares(X, Y):
    theta_hat = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)
    theta_hat[0] = np.math.e ** theta_hat[0]
    return theta_hat

def create_M(x, N, theta):
    M = np.zeros((len(func(x[0], theta)), len(func(x[0], theta))))
    for i in range(len(x)):
        M += 1.0/N * create_part_M(func(x[i], theta))
    return M

def create_part_M(fx):
    M = np.zeros((len(fx), len(fx)))
    for i in range(len(fx)):
        for j in range(len(fx)):
            M[i][j] = fx[i] * fx[j]
    return M

def create_D(M):
    return np.linalg.inv(M)

def d(x, D, newx, theta_hat):
    return np.dot(np.dot(func(x, theta_hat), D), func(newx, theta_hat).T)

def Delta(x, D, N, newx, theta_hat):
    return 1./float(N) * (d(newx, D, newx, theta_hat) - d(x, D, x, theta_hat))\
           - 1./float(N)**2 * (d(x, D, x, theta_hat) * d(newx, D, newx, theta_hat) - d(x, D, newx, theta_hat)**2)

def create_X(plan):
    return np.array(list(map(lambda x: [1.0, np.math.log(x[0]), np.math.log(x[1]), np.math.log(x[2])], plan)))

def create_Y(plan, theta, p = 0.2):
    Y = list(map(lambda x: theta[0] * x[0]**theta[1] * x[1]**theta[2] * x[2]**theta[3], plan))
    Y = list(map(lambda y: y + random.normalvariate(0, p * y), Y))
    return Y

# find max delta for one point 
def find_max_for_X(x, D, N, grid, theta_hat):
    maxdot = [grid[0], grid[0], grid[0]]
    maxvalue = Delta(x, D, N, maxdot, theta_hat)
    for x1 in grid:
        for x2 in grid:
          for x3 in grid:
            value = Delta(x, D, N, [x1, x2, x3], theta_hat)
            if value > maxvalue:
                maxvalue = value
                maxdot = [x1, x2, x3]
    return [maxvalue, maxdot]

# find max for each point in the plan
def find_max(X, D, N, grid, theta_hat):
    listofmax = [find_max_for_X(x, D, N, grid, theta_hat) for x in X]
    return [*max(listofmax), listofmax.index(max(listofmax))]

N = 5
grid = np.linspace(0.1, 10, 100)
plan = make_plan(grid = grid)
theta = [0.1, 0.2, 0.1, 0.3]
Y = create_Y(plan, theta)
Y = np.log(Y)
X = create_X(plan)

theta_hat = calculate_least_squares(X, Y)
plan_init = make_plan(grid = grid, m = N)
plan_opt = make_opt(theta_hat, plan_init, grid, N)
rss_avg, norm_avg_hat = do_exp(theta, plan_opt)

plan_init = make_plan(grid = grid, m = N)
rss_avg, norm_avg_hat = do_exp(theta, plan_init)

det 3.1134623364627236e-07
0


KeyboardInterrupt: ignored